In [1]:
!gpustat

vclab-gpuserver-57          Tue Dec 16 01:31:44 2025  550.78
[0] NVIDIA GeForce RTX 3090 | 28'C,   0 % |   324 / 24576 MB |
[1] NVIDIA GeForce RTX 3090 | 31'C,   0 % |   324 / 24576 MB |
[2] NVIDIA GeForce RTX 3090 | 26'C,   0 % |   324 / 24576 MB |
[3] NVIDIA GeForce RTX 3090 | 26'C,   0 % |   324 / 24576 MB |
[4] NVIDIA GeForce RTX 3090 | 27'C,   0 % |   324 / 24576 MB |
[5] NVIDIA GeForce RTX 3090 | 29'C,   0 % |   324 / 24576 MB |
[6] NVIDIA GeForce RTX 3090 | 26'C,   0 % |   324 / 24576 MB |
[7] NVIDIA GeForce RTX 3090 | 27'C,   0 % |   324 / 24576 MB |


In [2]:
# 数据库引入

import os
os.environ["CUDA_VISIBLE_DEVICES"] ='1'
device='cuda:0'

import sys
import ast
import csv
import math
import time
import wfdb
import copy
import torch
import random
import numpy as np
import pandas as pd
from utils import *
import torch.nn as nn
from tqdm import tqdm
import sklearn.metrics
from scipy import signal
import torch.nn.init as init
from collections import deque  
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchsummary import summary
import torch.autograd as autograd
from torch.autograd import Variable
from collections import defaultdict
import torch.backends.cudnn as cudnn
from torch.optim import lr_scheduler
from torch.nn import functional as F
from models.DG_models import generate_model
from datasets import train_loader,val_loader,test_loader
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer

loss_fn_clf=nn.CrossEntropyLoss()
train_val_test_split=[0.8,0.2,1] # 1 for inter
batch_size=128
class_num=3
smote=False

In [3]:
# 加载数据
train_data_list=['L','B','C','A','N']

train_datasloader_instance=train_loader(dataname=train_data_list,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,smote=smote,maple=False,class_num=class_num)
train_datasloader=train_datasloader_instance.loader()

val_data_list_A=['A']
val_datasloader_A=val_loader(dataname=val_data_list_A,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()


val_data_list_B=['B']
val_datasloader_B=val_loader(dataname=val_data_list_B,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()


val_data_list_C=['C']
val_datasloader_C=val_loader(dataname=val_data_list_C,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()


# val_data_list_H=['H']
# val_datasloader_H=val_loader(dataname=val_data_list_H,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()


val_data_list_L=['L']
val_datasloader_L=val_loader(dataname=val_data_list_L,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()


val_data_list_N=['N']
val_datasloader_N=val_loader(dataname=val_data_list_N,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()


test_H_data_list=['H']
test_H_datasloader=test_loader(dataname=test_H_data_list,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()



# test_T_data_list=['T_3cls']
# test_T_datasloader=test_loader(dataname=test_T_data_list,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()

# test_R_data_list=['R_3cls']
# test_R_datasloader=test_loader(dataname=test_R_data_list,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()



train data L
train data size : 16824
train label class :  [13511.  1158.  2155. 16824.     0.     0.     0.     0.]
train data B
train data size : 5501
train label class :  [ 742.  859. 3900.    0. 5501.    0.    0.    0.]
train data C
train data size : 24194
train label class :  [ 6719.  2039. 15436.     0.     0. 24194.     0.     0.]
train data A
train data size : 8274
train label class :  [1393.  480. 6401.    0.    0.    0. 8274.    0.]
train data N
train data size : 8516
train label class :  [1462. 1769. 5285.    0.    0.    0.    0. 8516.]
total train data shape =  (63309, 1000, 1)
total train label shape =  (63309, 8)
total train data class =  [23827.  6305. 33177. 16824.  5501. 24194.  8274.  8516.]
val data A
val data size : 2069
val label class :  [ 359.   90. 1620.]
total val data shape =  (2069, 1000, 1)
total val label shape =  (2069, 3)
total val data class =  [ 359.   90. 1620.]
val data B
val data size : 1376
val label class :  [176. 239. 961.]
total val data shape =  

In [4]:
# 生成模型定义，包括两个encoder和一个decoder

class AdaptiveInstanceNorm2d(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super(AdaptiveInstanceNorm2d, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        # weight and bias are dynamically assigned
        self.weight = None
        self.bias = None
        # just dummy buffers, not used
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))

    def forward(self, x):
        assert self.weight is not None and self.bias is not None, "Please assign weight and bias before calling AdaIN!"
        b, c = x.size(0), x.size(1)
        running_mean = self.running_mean.repeat(b)
        running_var = self.running_var.repeat(b)

        # Apply instance norm
        x_reshaped = x.contiguous().view(1, b * c, *x.size()[2:])
        out = F.batch_norm(
            x_reshaped, running_mean, running_var, self.weight, self.bias,
            True, self.momentum, self.eps)

        return out.view(b, c, *x.size()[2:])

    def __repr__(self):
        return self.__class__.__name__ + '(' + str(self.num_features) + ')'

    
class Decoder(nn.Module):
    def __init__(self,):
        super(Decoder, self).__init__()

        # dec
        self.conv1=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn1=AdaptiveInstanceNorm2d(64)
        self.ac1=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv2=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn2=AdaptiveInstanceNorm2d(64)
        self.ac2=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv3=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn3=AdaptiveInstanceNorm2d(64)
        self.ac3=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        
        
        self.conv4=nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.bn4=AdaptiveInstanceNorm2d(32)
        self.ac4=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv5=nn.Conv1d(32, 32, kernel_size=3, padding=1)
        self.bn5=AdaptiveInstanceNorm2d(32)
        self.ac5=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv6=nn.Conv1d(32, 16, kernel_size=3, padding=1)
        self.bn6=AdaptiveInstanceNorm2d(16)
        self.ac6=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv7=nn.Conv1d(16, 16, kernel_size=3, padding=1)
        self.bn7=AdaptiveInstanceNorm2d(16)
        self.ac7=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv8=nn.Conv1d(16, 8, kernel_size=3, padding=1)
        self.bn8=AdaptiveInstanceNorm2d(8)
        self.ac8=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv9=nn.Conv1d(8, 8, kernel_size=3, padding=1)
        self.ac9=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv10=nn.Conv1d(8, 1, kernel_size=1)
        
    def forward(self, x):
        
        x=self.ac1(x+self.bn1(self.conv1(x)))
        x=self.ac2(x+self.bn2(self.conv2(x)))
        x=self.ac3(x+self.bn3(self.conv3(x)))
        x=self.ac4(self.bn4(self.conv4(x)))
        x=self.ac5(self.bn5(self.conv5(x)))
        x=self.ac6(self.bn6(self.conv6(x)))
        x=self.ac7(self.bn7(self.conv7(x)))
        x=self.ac8(self.bn8(self.conv8(x)))
        x=self.conv10(self.ac9(self.conv9(x)))
        
        return x


class ECG_gen(nn.Module):  
    def __init__(self,):  
        super(ECG_gen, self).__init__()  
        
        # cont_enc
        
        self.conv1=nn.Conv1d(1, 8, kernel_size=3, padding=1)
        self.bn1=nn.InstanceNorm1d(8)
        self.ac1=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv2=nn.Conv1d(8, 8, kernel_size=3, padding=1)
        self.bn2=nn.InstanceNorm1d(8)
        self.ac2=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv3=nn.Conv1d(8, 16, kernel_size=3, padding=1)
        self.bn3=nn.InstanceNorm1d(16)
        self.ac3=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv4=nn.Conv1d(16, 16, kernel_size=3, padding=1)
        self.bn4=nn.InstanceNorm1d(16)
        self.ac4=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv5=nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.bn5=nn.InstanceNorm1d(32)
        self.ac5=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv6=nn.Conv1d(32, 32, kernel_size=3, padding=1)
        self.bn6=nn.InstanceNorm1d(32)
        self.ac6=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv7=nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.bn7=nn.InstanceNorm1d(64)
        self.ac7=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv8=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn8=nn.InstanceNorm1d(64)
        self.ac8=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        
        
        self.conv9=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn9=nn.InstanceNorm1d(64)
        self.ac9=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv10=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn10=nn.InstanceNorm1d(64)
        self.ac10=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv11=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn11=nn.InstanceNorm1d(64)
        self.ac11=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        # style_enc
        
        self.conv_sty_1=nn.Conv1d(1, 8, kernel_size=3, padding=1)
        self.ac_sty_1=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv_sty_2=nn.Conv1d(8, 16, kernel_size=3, padding=1)
        self.ac_sty_2=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv_sty_3=nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.ac_sty_3=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv_sty_4=nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.ac_sty_4=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv_sty_5=nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.ac_sty_5=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv_sty_6=nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.ac_sty_6=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.pool_sty=nn.AdaptiveAvgPool1d(1)
        self.fc_sty=nn.Conv1d(256, 8, kernel_size=1, padding=0)
        # bs,8,1
        
        self.dec=Decoder()
        
        self.mlp_1=nn.Linear(8, 256, bias=True)
        self.mlp_2=nn.LeakyReLU(0.01, inplace=True)
        self.mlp_3=nn.Linear(256, 256, bias=True)
        self.mlp_4=nn.LeakyReLU(0.01, inplace=True)
        self.mlp_5=nn.Linear(256, self.get_num_adain_params(self.dec), bias=True)
        
        
    def encode(self, x):  # bs,1000,1  x1是特征，x2是风格
        
        x=x.transpose(-1,-2) # bs,1,1000
        
        xc=self.conv1(x)
        xc=self.bn1(xc)
        xc=self.ac1(xc)
        xc=self.ac2(self.bn2(self.conv2(xc)))
        xc=self.ac2(self.bn2(self.conv2(xc)))
        xc=self.ac3(self.bn3(self.conv3(xc)))
        xc=self.ac4(self.bn4(self.conv4(xc)))
        xc=self.ac5(self.bn5(self.conv5(xc)))
        xc=self.ac6(self.bn6(self.conv6(xc)))
        xc=self.ac7(self.bn7(self.conv7(xc)))
        xc=self.ac8(self.bn8(self.conv8(xc)))
        xc=self.ac9(xc+self.bn9(self.conv9(xc)))
        xc=self.ac10(xc+self.bn10(self.conv10(xc)))
        content=self.ac11(xc+self.bn11(self.conv11(xc)))   # bs,64,1000

        
        
        xs=self.conv_sty_1(x)
        xs=self.ac_sty_1(xs)
        xs=self.conv_sty_2(xs)
        xs=self.ac_sty_2(xs)
        xs=self.conv_sty_3(xs)
        xs=self.ac_sty_3(xs)
        xs=self.conv_sty_4(xs)
        xs=self.ac_sty_4(xs)
        xs=self.conv_sty_5(xs)
        xs=self.ac_sty_5(xs)
        xs=self.conv_sty_6(xs)
        xs=self.ac_sty_6(xs)
        xs=self.pool_sty(xs)
        style=self.fc_sty(xs)
        
        return content, style
    
    def decode(self,content, style):  # x1逆向操作，x2应用到自适应归一化层
        
        adain_params = self.mlp_1(style.view(style.size(0), -1))
        adain_params = self.mlp_2(adain_params)
        adain_params = self.mlp_3(adain_params)
        adain_params = self.mlp_4(adain_params)
        adain_params = self.mlp_5(adain_params)
        self.assign_adain_params(adain_params, self.dec)
        x=self.dec(content)
        x=x.transpose(-1,-2)
        
        return x
        
        
    def assign_adain_params(self, adain_params, model):
        for m in model.modules():
            if m.__class__.__name__ == "AdaptiveInstanceNorm2d":
                mean = adain_params[:, :m.num_features]
                std = adain_params[:, m.num_features:2*m.num_features]
                m.bias = mean.contiguous().view(-1)
                m.weight = std.contiguous().view(-1)
                if adain_params.size(1) > 2*m.num_features:
                    adain_params = adain_params[:, 2*m.num_features:]
        
        
    def get_num_adain_params(self,model):
        num_adain_params = 0
        for m in model.modules():
            if m.__class__.__name__ == "AdaptiveInstanceNorm2d":
                num_adain_params += 2*m.num_features
        return num_adain_params
        
    def forward(self, images):
        # reconstruct an image
        content, style_fake = self.encode(images)
        images_recon = self.decode(content, style_fake)
        return images_recon
    
    
    

In [5]:
# 加载模型的流程定义

def load_munit_model(model_path):

    return MUNITModelOfNatVar(model_path).to(device)



class MUNITModelOfNatVar(nn.Module):
    def __init__(self, fname: str):

        super(MUNITModelOfNatVar, self).__init__()

        self._fname = fname
        self._gen_A, self._gen_B = self.__load()
        self.delta_dim = 8

    def forward(self, x, delta):

        orig_content, _ = self._gen_A.encode(x)
        orig_content = orig_content.clone().detach().requires_grad_(False)
        x_out = self._gen_B.decode(orig_content, delta)

        return x_out

    def __load(self):
        """Load MUNIT model from file."""

        def load_munit(fname, letter):
            gen = ECG_gen()
            gen.load_state_dict(torch.load(fname)[letter])
            return gen.eval()

        gen_A = load_munit(self._fname, 'a')
        gen_B = load_munit(self._fname, 'b')

        return gen_A, gen_B     # original order
        


In [6]:
# 算法总模型定义


class Algorithm(torch.nn.Module):

    def __init__(self,):
        super(Algorithm, self).__init__()

    def update(self, in_data_, targets_, unlabeled=None):
        raise NotImplementedError

    def predict(self, x):
        raise NotImplementedError

class ERM(Algorithm):

    def __init__(self, ):
        super(ERM, self).__init__()


        self.network = generate_model(base_model='cnn_Ag',input_channels=1, num_classes=3,DG_method=None,domain_classes=6,distill=False)
        self.optimizer = torch.optim.Adam(self.network.parameters(),lr=1e-3,weight_decay=0)

    def update(self, in_data_, targets_, unlabeled=None):
        
        loss = F.cross_entropy(self.predict(in_data_), targets_)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return {'loss': loss.item()}

    def predict(self, x):
        return self.network(x)

class MBDG_Base(ERM):
    def __init__(self, ):
        super(MBDG_Base, self).__init__()
    
        # self.G = load_munit_model('AF_DG_OODG/GAN/gen_00600000.pt')
        self.G = load_munit_model('AF_DG_OODG/GANMBDG/gen_00600000.pt')

    @staticmethod
    def kl_div(dist1, dist2):
        return F.kl_div(torch.log(dist2), dist1, reduction='batchmean')

    def predict(self, x):
        return self.network(x)

    @torch.no_grad()
    def generate_images(self, images):
        delta = torch.randn(images.size(0), self.G.delta_dim).to(device).requires_grad_(False)
        return self.G(images, delta)

    def calc_dist_reg(self, x, clean_output):
        mb_images = self.generate_images(x)
        mb_output = F.softmax(self.predict(mb_images), dim=1)
        return self.kl_div(F.softmax(clean_output, dim=1), mb_output)

    @staticmethod
    def relu(x):
        return x if x > 0 else torch.tensor(0).to(device)

class MBDG(MBDG_Base):

    def __init__(self, ):
        super(MBDG, self).__init__()
        self.dual_var = torch.tensor(1.0).to(device).requires_grad_(False)

    def update(self, in_data_, targets_, unlabeled=None):

        clean_output = self.predict(in_data_)
        clean_loss = F.cross_entropy(clean_output, targets_)
        dist_reg = self.calc_dist_reg(in_data_, clean_output)

        loss = clean_loss + self.dual_var * dist_reg

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        const_unsat = dist_reg.detach() - 0.025
        self.dual_var = self.relu(self.dual_var + 0.05 * const_unsat)

        return {'loss': loss.item(), 'dist_reg': dist_reg.item(), 'dual_var': self.dual_var.item()}
    
    def forward(self, x):
        return self.predict(x)


In [7]:
from deepspeed.profiling.flops_profiler import get_model_profile
from deepspeed.accelerator import get_accelerator

torch.multiprocessing.set_start_method('spawn', force=True)

with get_accelerator().device(0):
    cls_model=MBDG().to(device)
    cls_model.eval()

    val_datasloader_A_iter = iter(val_datasloader_A)
    in_data_, targets_ = next(val_datasloader_A_iter)
    input_shape = tuple(in_data_.shape)
    
    flops, macs, params = get_model_profile(model=cls_model, # model
                                    input_shape=input_shape, # input shape to the model. If specified, the model takes a tensor with this shape as the only positional argument.
                                    args=None, # list of positional arguments to the model.
                                    kwargs=None, # dictionary of keyword arguments to the model.
                                    print_profile=False, # prints the model graph with the measured profile attached to each module
                                    detailed=True, # print the detailed profile
                                    module_depth=-1, # depth into the nested modules, with -1 being the inner most modules
                                    top_modules=1, # the number of top modules to print aggregated profile
                                    warm_up=10, # the number of warm-ups before measuring the time of each module
                                    as_string=True, # print raw numbers (e.g. 1000) or as human-readable strings (e.g. 1k)
                                    output_file=None, # path to the output file. If None, the profiler prints to stdout.
                                    ignore_modules=None) # the list of modules to ignore in the profiling
    print(f'params of model: {params}')

/home/xiaojianlin/miniconda3/envs/clkg_ecp/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/xiaojianlin/miniconda3/envs/clkg_ecp/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


base_model=cnn_Ag


/tmp/ipykernel_3592912/2002937344.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  gen.load_state_dict(torch.load(fname)[letter])


params of model: 11.46 M
